In [23]:
'''
Author        : Aditya Jain
Date started  : 9th September, 2021
About         : This script is used for the self-supervision idea
'''
import torchvision.models as models
import torch
import cv2
import numpy as np
from torchvision import transforms, utils
from PIL import Image

In [35]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
resnet_mod = models.resnet50(pretrained=True)

cpu


In [64]:
def frames_from_video(video_path, des_fps, img_size):
    ''' 
    Given a video, prepares the final downsized concatenated image for training
    Arguments:
    video_path: the file path of video
    n_frames  : no of frames to sample from the video at equal intervals
    down_f    : downscaling factor for the image
    '''
    
    vidcap      = cv2.VideoCapture(video_path)     
    fps         = vidcap.get(cv2.CAP_PROP_FPS)           #  FPS of the video      
    frame_count = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)   #  total frame count
    total_sec   = frame_count/fps
    sec         = 0
    n_frames    = total_sec*des_fps
    time_sec    = total_sec/n_frames                     # the video will be sampled after every time_sec
    final_img   = []
        
    while sec < total_sec:        
        vidcap.set(cv2.CAP_PROP_POS_MSEC, sec * 1000)    # setting which frame to get        
        success, image = vidcap.read()

        if success:
            transformer = transforms.Compose([
                            transforms.Resize((img_size, img_size)),              # resize the image to 224x224 
                            transforms.ToTensor()])
            image       = Image.fromarray(image)
            image       = transformer(image)
            
            final_img.append(image)
            
        sec += time_sec        
        
    return torch.Tensor(final_img.numpy())


In [65]:
img_size   = 224
vid_path   = 'dog.mp4'
target_fps = 5

frame_list = frames_from_video(vid_path, target_fps, img_size)

AttributeError: 'list' object has no attribute 'numpy'

In [39]:
test_image = torch.unsqueeze(frame_list[0], 0).to(device)
prediction = resnet_mod(test_image)
values, predict_indx = torch.topk(prediction, 5)

In [ ]:
print(frame_list.shape)